In [2]:
import numpy as np
from scipy.optimize import root
from scipy.integrate import odeint
from scipy.linalg import solve_continuous_are as scare
import gym, acme_gym

#These are the functions from Brooke's Inverted Pendulum lab
def linearized_init(M, m, l, q1, q2, q3, q4, r):
    '''
    Parameters:
    ----------
    M, m: floats
    masses of the rickshaw and the present
    l   : float
    length of the rod
    q1, q2, q3, q4, r : floats
    relative weights of the position and velocity of the rickshaw, 
    the angular displacement theta and the change in theta, and the control
    Return
    -------
    A : ndarray of shape (4,4)
    B : ndarray of shape (4,1)
    Q : ndarray of shape (4,4)
    R : ndarray of shape (1,1)
    '''
    g = 9.8
    A = np.zeros((4,4))
    A[0,1] = 1
    A[1,2] = m*g/M
    A[2,3] = 1
    A[3,2] = g/(M*l)*(M+m)
    B = np.zeros((4,1))
    B[1] = 1/M
    B[3] = 1/(M*l)
    Q = np.diag([q1, q2, q3, q4])
    R = np.array([[r]])
    return A, B, Q, R

def find_P(A, B, Q, R):
    '''
    Parameters:
    ----------
    A, Q    : ndarrays of shape (4,4)
    B       : ndarray of shape (4,1)
    R       : ndarray of shape (1,1)
    Returns
    -------
    P       : the matrix solution of the Riccati equation
    '''
    def fun(P):
        P = P.reshape((4,4))
        root = P@A+A.T@P+Q-1/R[0]*(P@B@B.T@P)
        return root.reshape(16)
    P0 = np.ones(16)
    P = root(fun,P0).x.reshape((4,4))
    return P

def rickshaw(tv, X0, A, B, Q, R, P):
    '''
    Parameters:
    ----------
    tv  : ndarray of time values, with shape (n+1,)
    X0  : Initial conditions on state variables
    A, Q: ndarrays of shape (4,4)
    B   : ndarray of shape (4,1)
    R   : ndarray of shape (1,1)
    P   : ndarray of shape (4,4)
    Returns
    -------
    Z : ndarray of shape (n+1,4), the state vector at each time
    U : ndarray of shape (n+1,), the control values
    '''
    
    func = lambda z,t: (A - 1/R[0]*B@B.T@P)@z.T
    Z = odeint(func, X0, tv)
    BP = B.T@P
    U = -1/R[0]*(BP)@Z.T
    return Z,U  

def stabilize(M, m, l, q1, q2, q3, q4, r, X0, tf, step):
    A, B, Q, R = linearized_init(M, m, l, q1, q2, q3, q4, r)
    P = scare(A, B, Q, R)
    tv = np.linspace(0,tf,step)
    Z, U = rickshaw(tv,X0,A, B, Q, R, P)
    #Z is state vector
    #U is control values
    return Z,U
    
def cartpole(environment, init_state, tol=1e-2, disp=False, N = 300):
    env = environment
    obs = init_state
    
    control = []
    
    '''
    This function will run the cartpole problem using the environment and initial conditions provided.
    Do NOT call env.reset().
    Run whatever system you desire to make sure the state values fall under the tolerance level.
    Convergence is considered reached once numpy.linalg.norm(obs[1:]) < tol, where we ignore the x position of the cart.
    You will need to return the sequence of controls that brings the cartpole system into vertical stability.
    Make sure to quit after N iterations, or convergence is reached, whichever occurs first.
    Remember you are being graded against other teams' step counts, so you want to end the system updates as soon as possible.
    
    Parameters:
        environment (CartPole object): The cartpole environment as described in gym and acme_gym.
        init_state (tuple): The initial state with [x, x', θ, θ'].
        tol (float): The tolerance to be reached before the cartpole problem is considered converged.
        disp (bool): If True, render the image.
        N (int): The max number of iterations.
    
    Returns:
        (list): A list of control values. These will be tested on the grader's end for convergence and time step assessment.
    
    '''
    env = gym.make('CartPoleContinuous-v0')
    T = round(6/0.02)
    # Initial state is drawn randomly, let the user pick a good starting point
    init_state = True
    if init_state:
        obs = env.reset()
        env.render()
        
    step = 500
    x0 = obs
    q1, q2, q3, q4 = 90000., 1., 90000., 1
    tf = .02*step
    r = 10. # Weight on the control, how do we know what it should be?
    M, m, l = 1, .1, 1
    Z, U = stabilize(M, m, l, q1, q2, q3, q4, r, x0, tf, step)
    U = U.T

    for i in range(step):
        #get a new obs every interation... are we supposed to include that in new calcs?
        control.append(U[0])
        obs, reward, state, info = env.step(np.array(U[0]))
        env.render()

        # Feedback
        x0 = np.array([obs[0],obs[1],-obs[2],-obs[3]])
        Z, U = stabilize(M, m, l, q1, q2, q3, q4, r, x0, tf, step)
        U = U.T
    
    env.close()
    return control

ModuleNotFoundError: No module named 'acme_gym'

In [ ]:
import gym, acme_gym
import numpy as np
import copy
from scipy import linalg
from tqdm import tqdm

tol = 1e-2
total = 200

render = True

num_steps = []

for i in tqdm(range(total)):
    env = gym.make("CartPoleContinuous-v0")
    observation = env.reset()
    env_copy = copy.deepcopy(env)
    
    # This is students' code
    control = cartpole(env_copy, observation, tol, render)
    
    for u in control:
        obs, reward, state, info = env.step(np.array([u]))
        
        if render:
            env.render()
    
    render = False
    
    if np.linalg.norm(obs[1:]) > tol:
        print("unsuccessful")
    
    num_steps.append(len(control))
    env.close()

print("Average number of steps after {} iterations: {}".format(total, np.mean(num_steps)))